In [91]:
import time
import datetime as dt
import pandas as pd
import snowflake.connector
from pypardot.client import PardotAPI, PardotAPIError
import snowflake.connector


In [92]:
PARDOT_EMAIL="segment_integrations@discoveryed.com"
PARDOT_USER_KEY="f5a1dc61d079e35d9a2066a4a8498c32"
PARDOT_SF_CONSUMER_KEY="3MVG9IHf89I1t8hpom1l0QzbTANHH.MOASIZ3yPPhu6hFI_uipXiYi7ku85yMWg_2gXxu5bzAyWCevOHp4jrf"
PARDOT_SF_CONSUMER_SECRET="BC6C2AA5D41DC0EBBBB3376CD329E92E66BD8D85DF85F9FAB9F22E86D857A29E"
PARDOT_SF_REFRESH_TOKEN="5Aep8618yVsldz6rZPMv4ouelrGoRNAfdaLSjo3.ILw4jdNGemJVI_MNYFoNkN9g39GUXqUEgzw3ylEVMjxAU3X"
PARDOT_BUSINESS_UNIT_ID="0Uv4P000000TNB0SAO"
PARDOT_API_VERSION=4
PARDOT_MAX_RESULT_COUNT = 200
SNOWFLAKE_ACCOUNT_IDENTIFIER="UK29315.us-east-1"
SNOWFLAKE_USER="PardotEtlUser@discoveryed.com"
SNOWFLAKE_PASS="p@rd0tEtlUser"

In [93]:
def get_client_pardot() -> PardotAPI:
    try:
        p = PardotAPI(
            email=PARDOT_EMAIL,
            user_key=PARDOT_USER_KEY,
            sf_consumer_key=PARDOT_SF_CONSUMER_KEY,
            sf_consumer_secret=PARDOT_SF_CONSUMER_SECRET,
            sf_refresh_token=PARDOT_SF_REFRESH_TOKEN,
            business_unit_id=PARDOT_BUSINESS_UNIT_ID,
            version=PARDOT_API_VERSION,
        )

        if not p.authenticate():
            p.campaigns.query(limit=1)
            # ^ Must call query to authenticate if authenticatation is unsuccessful

    except PardotAPIError as err:
        raise

    assert (
        p.sftoken != "dummy"
    ), "Pardot authentication still unsuccessful after querying"

    return p

def get_client_snowflake():
    return snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASS,
        account=SNOWFLAKE_ACCOUNT_IDENTIFIER,
        # authenticator="externalbrowser",
        warehouse="GENERAL_COMPUTE_WH",
        database="PROD_DATA_VAULT",
        schema="STAGE",
    )

In [94]:
p = get_client_pardot()

In [95]:
tables = [
'Campaign',
'Form',
'Tag',
'Opportunity',
'EmailClick',
'List',
'Account',
'TagObject',
"Prospect",
"VisitorActivity",
"Visitor",
"ListMembership",
"ProspectAccount"
]

tag_object_types = ["Email","Campaign","List"]

DICT_CONVERSION_PURAL = {
    "Account": "accounts",
    "EmailClick": "emailclicks",
    "Email": "emails",
    "VisitorActivity": "visitoractivities",
    "Campaign": "campaigns",
    "List": "lists",
    "Prospect": "prospects",
    "TagObject": "tagobjects",
    "Tag": "tags",
    "Opportunity": "opportunities",
    "ProspectAccounts": "prospectaccounts",
    "Form": "forms",
    "ListMembership": "listmemberships",
    "Visitor": "visitors",
    "ProspectAccount": "prospectaccounts",
}


In [96]:
def helper_to_camelCase(s: str) -> str:
    "Return the same string with the first letter lowercase"
    return str(s[0].lower() + s[1:])

In [97]:
expected_results = {}
for data_type in tables:
    try:
        time.sleep(3)
        print(data_type)
        data_client = getattr(p, DICT_CONVERSION_PURAL[data_type])
        if not hasattr(data_client, "query"):
            # If the data_client doesn't have a query method, try the "read" method
            data = data_client.read()[helper_to_camelCase(data_type)]
            if isinstance(data, dict):
                # Assume only one record
                expected_results[data_type]=1
            elif isinstance(data, list):
                expected_results[data_type]=len(data)
        else:
            if data_type=="TagObject":
                counter = 0
                for tag_type in tag_object_types:
                    time.sleep(2)
                    data_raw = data_client.query(limit=1,type=tag_type)
                    counter = counter + data_raw["total_results"]
                expected_results[data_type] = counter
            else:
                data_raw = data_client.query(limit=1)
                expected_results[data_type]=data_raw["total_results"]
    except:
        print(f"{data_type} failed")
        raise



Campaign
Form
Tag
Opportunity
EmailClick


In [81]:
ctx = get_client_snowflake()
cs = ctx.cursor()
name_table_snowflake = "PARDOT_OPPORTUNITY"
SF_SCHEMA = "PROD_DATA_VAULT"


tables = ['Campaign',
'Form',
'Tag',
'Opportunity',
'EmailClick',
'List',
'Account','TagObject',"Email","Prospect","VisitorActivity","Visitor","ListMembership","ProspectAccount"]

queries = []
for table in tables:
    if table == "TagObject":
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct tag_object_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct tag_object_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    elif table == "VisitorActivity":
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct visitor_activity_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct visitor_activity_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    else:
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct {table}_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct {table}_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    queries.append(sql_template)

query_join = ' union all '.join(queries)

email_query = """select 
        count(distinct a.list_email_id) as expected
        from PARDOT_VISITORACTIVITY a
        where a.email_id is not null and a.list_email_id is not null"""

cs.execute(query_join)
data =  cs.fetchall()  
columns = [f'{col[0]}' for col in cs.description]


sf_data = pd.DataFrame(data,columns=columns)

try:
    cs.execute(email_query)
    email_expected =  cs.fetchone()[0]
finally:
    cs.close()
ctx.close()
sf_data

sf_data.loc[sf_data["DATA_OBJECT"]=="Email","EXPECTED"] = email_expected

In [85]:
for d_obj, exected in expected_results.items():
    sf_data.loc[sf_data["DATA_OBJECT"]==d_obj,"EXPECTED"] = exected

sf_data.loc[sf_data["EXPECTED"]<sf_data["COUNT_STAGE_DISTINCT"],"ISSUE"] = "EXPECTED LOWER"
sf_data.loc[sf_data["EXPECTED"]>sf_data["COUNT_STAGE_DISTINCT"],"ISSUE"] = "DATA MISSING"
sf_data.loc[sf_data["EXPECTED"]==sf_data["COUNT_STAGE_DISTINCT"],"ISSUE"] = "CORRECT"
sf_data


,DATA_OBJECT,COUNT_MARKET_DISTINCT,COUNT_MARKET_TOTAL,COUNT_STAGE_DISTINCT,COUNT_STAGE_TOTAL,EXPECTED,ISSUE
0,Campaign,551,551,552,1103,552.0,CORRECT
1,Form,7,7,14,21,14.0,CORRECT
2,Tag,2,2,174,176,174.0,CORRECT
3,Opportunity,8499,8499,8499,10508,8499.0,CORRECT
4,EmailClick,1141682,1141684,1142962,1142965,1142949.0,EXPECTED LOWER
5,List,1794,1794,1800,1800,1798.0,EXPECTED LOWER
6,Account,1,1,1,2,1.0,CORRECT
7,TagObject,3469,6932,3485,3485,3491.0,DATA MISSING
8,Email,0,0,0,0,2091.0,DATA MISSING
9,Prospect,0,0,0,0,4714195.0,DATA MISSING


In [90]:
# If required, download and push this through the notebook
import requests
headers = {
    "content-type": "application/json",
    "Authorization": f"Bearer {p.sftoken}",
    "Pardot-Business-Unit-Id": PARDOT_BUSINESS_UNIT_ID,
}
PARDOT_API_VERSION = 4
PARDOT_URL_API = f"https://pi.pardot.com/api/export/version/{PARDOT_API_VERSION}"
ID = 4480

response = requests.get(
    f"{PARDOT_URL_API}/do/read/id/{ID}",
            headers=headers,
            params=(("format", "json"),),
        )

response.json()

{'@attributes': {'stat': 'ok', 'version': 1},
 'export': {'id': 4480,
  'state': 'Processing',
  'object': 'visitorActivity',
  'fields': ['id',
   'campaign_id',
   'created_at',
   'updated_at',
   'custom_redirect_id',
   'details',
   'email_id',
   'email_template_id',
   'file_id',
   'form_id',
   'form_handler_id',
   'landing_page_id',
   'list_email_id',
   'multivariate_test_variation_id',
   'opportunity_id',
   'paid_search_ad_id',
   'prospect_id',
   'site_search_query_id',
   'type',
   'type_name',
   'visit_id',
   'visitor_id',
   'visitor_page_view_id'],
  'procedure': {'name': 'filter_by_updated_at',
   'arguments': {'updated_after': '2021-01-01 00:00:00',
    'updated_before': '2021-12-31 00:00:00'}},
  'isExpired': False,
  'createdAt': '2022-02-01 16:06:45',
  'updatedAt': '2022-02-01 17:34:55'}}